In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                            VALID:                \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG     LOSS   PPL     NLL   
5000     72   35   99  522.59  837  519.67  2.73  0.19  1001.64  1795  997.31   
10000    49   71   59  522.01  832  518.58  3.26  0.17   950.22  1264  945.97   
15000    43  107   19  522.46  835  518.95  3.37  0.14   943.20  1188  938.64   
20000    51  142  119  521.97  833  518.30  3.50  0.17   949.87  1246  945.04   
25000    49  178   79  521.41  828  517.62  3.62  0.18   944.04  1192  939.02   
30000    42  214   39  521.31  827  517.50  3.63  0.17   945.90  1211  941.28   
35000    38  249  139  521.67  830  517.94  3.57  0.16   949.47  1255  945.80   
40000    40  285   99  521.30  827  517.53  3.61  0.16   944.88  1207  940.16   
45000    46  321   59  520.96  823  517.13  3.65  0.17   944.00  1189  939.18   
50000    49  357   19  520.70  819  516.82  3.70  0.18   943.62  1183  938.68   
55000    48  392  119  520.35  816  516.42  3.74  0.18   945.78  1205  940.69   
60000    48  428   79  520.06  813  516.10  3.78  0.18   943.96  1189  938.85   
65000    50  464   39  519.88  810  515.89  3.81  0.18   943.58  1180  938.39   
70000    51  499  139  519.69  808  515.69  3.84  0.18   942.74  1176  937.52   
75000    51  535   99  519.46  806  515.44  3.87  0.18   943.31  1180  938.16   
80000    53  571   59  519.30  804  515.26  3.89  0.17   942.61  1169  937.41   
85000    51  607   19  519.18  802  515.13  3.91  0.17   942.49  1165  937.23   
90000    51  642  119  519.00  801  514.94  3.93  0.17   943.48  1176  938.15   
95000    49  678   79  518.85  799  514.77  3.95  0.16   942.84  1169  937.49   
100000   50  714   39  518.76  798  514.68  3.96  0.16   942.93  1167  937.52   
105000   50  749  139  518.67  797  514.58  3.98  0.16   943.01  1172  937.66   
110000   49  785   99  518.54  796  514.43  3.99  0.16   943.09  1172  937.77   
115000   50  821   59  518.46  795  514.34  4.00  0.15   943.17  1169  937.81   
120000   49  857   19  518.40  794  514.27  4.02  0.15   943.17  1168  937.80   
125000   49  892  119  518.29  793  514.16  4.03  0.15   943.97  1177  938.54   
130000   49  928   79  518.20  792  514.06  4.04  0.15   943.59  1173  938.16   
135000   48  964   39  518.16  791  513.98  4.05  0.14   943.83  1173  938.35   
140000   48  999  139  518.11  790  513.91  4.06  0.14   944.00  1179  938.60   

                     TEST:       SPEC:             HIER:        
          KL   REG    LOSS   PPL     1     2     3 CHILD OTHER  
5000    4.17  0.17  604.89  1133  0.15  0.28  0.38  0.60  0.43  
10000   4.13  0.12  582.45   946  0.21  0.38  0.34  0.47  0.33  
15000   4.48  0.08  580.47   937  0.22  0.31  0.41  0.43  0.29  
20000   4.62  0.22  580.47   937  0.20  0.28  0.32  0.54  0.41  
25000   4.82  0.20  580.47   937  0.22  0.33  0.27  0.51  0.42  
30000   4.50  0.12  580.47   937  0.21  0.31  0.38  0.42  0.35  
35000   3.58  0.10  580.47   937  0.16  0.27  0.22  0.54  0.45  
40000   4.55  0.17  580.47   937  0.20  0.32  0.32  0.45  0.38  
45000   4.57  0.24  580.47   937  0.21  0.31  0.26  0.50  0.46  
50000   4.72  0.21  576.88   876  0.23  0.28  0.28  0.50  0.46  
55000   4.89  0.19  576.88   876  0.22  0.29  0.30  0.50  0.44  
60000   4.94  0.17  576.88   876  0.23  0.32  0.28  0.47  0.43  
65000   5.03  0.16  576.61   871  0.23  0.29  0.30  0.49  0.44  
70000   5.07  0.15  576.52   872  0.24  0.28  0.32  0.48  0.42  
75000   5.01  0.14  576.52   872  0.24  0.31  0.31  0.48  0.41  
80000   5.07  0.13  576.35   867  0.24  0.30  0.31  0.48  0.43  
85000   5.13  0.12  576.18   867  0.24  0.27  0.32  0.48  0.43  
90000   5.22  0.12  576.18   867  0.23  0.28  0.32  0.48  0.42  
95000   5.24  0.11  576.18   867  0.24  0.31  0.31  0.46  0.41  
100000  5.31  0.11  576.18   867  0.24  0.29  0.31  0.48  0.43  
105000  5.25  0.10  576.18   867  0.24  0.28  0.32  0.48  0.43  
110000  5.23  0.10  576.18   867  0.24  0.30  0.31  0.47  0.41  
115000  5.26  0.09  576.18   867

0 R: 1.000 P: 0.142 space use write new article one know university please also
   1 R: 0.034 P: 0.004 use file window program image get version run server available
     11 R: 0.030 P: 0.030 key use chip encryption system one privacy security clipper algorithm
   2 R: 0.190 P: 0.017 use drive get card one problem work write system disk
     21 R: 0.137 P: 0.137 write get people government article one think right make know
     22 R: 0.036 P: 0.036 game team play go year player get win write season
   3 R: 0.634 P: 0.084 get write article one car go like use take think
     31 R: 0.549 P: 0.549 one god say write people think know make see believe
{0: [2, 3, 4], 2: [21], 3: [31, 32], 4: [41]}


TRAIN:                            VALID:                \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG     LOSS   PPL     NLL   
5000     72   35   99  522.59  837  519.67  2.73  0.19  1001.64  1795  997.31   
10000    49   71   59  522.01  832  518.58  3.26  0.17   950.22  1264  945.97   
15000    43  107   19  522.46  835  518.95  3.37  0.14   943.20  1188  938.64   
20000    51  142  119  521.97  833  518.30  3.50  0.17   949.87  1246  945.04   
25000    49  178   79  521.41  828  517.62  3.62  0.18   944.04  1192  939.02   
30000    42  214   39  521.31  827  517.50  3.63  0.17   945.90  1211  941.28   
35000    38  249  139  521.67  830  517.94  3.57  0.16   949.47  1255  945.80   
40000    40  285   99  521.30  827  517.53  3.61  0.16   944.88  1207  940.16   
45000    46  321   59  520.96  823  517.13  3.65  0.17   944.00  1189  939.18   
50000    49  357   19  520.70  819  516.82  3.70  0.18   943.62  1183  938.68   
55000    48  392  119  520.35  816  516.42  3.74  0.18   945.78  1205  940.69   
60000    48  428   79  520.06  813  516.10  3.78  0.18   943.96  1189  938.85   
65000    50  464   39  519.88  810  515.89  3.81  0.18   943.58  1180  938.39   
70000    51  499  139  519.69  808  515.69  3.84  0.18   942.74  1176  937.52   
75000    51  535   99  519.46  806  515.44  3.87  0.18   943.31  1180  938.16   
80000    53  571   59  519.30  804  515.26  3.89  0.17   942.61  1169  937.41   
85000    51  607   19  519.18  802  515.13  3.91  0.17   942.49  1165  937.23   
90000    51  642  119  519.00  801  514.94  3.93  0.17   943.48  1176  938.15   
95000    49  678   79  518.85  799  514.77  3.95  0.16   942.84  1169  937.49   
100000   50  714   39  518.76  798  514.68  3.96  0.16   942.93  1167  937.52   
105000   50  749  139  518.67  797  514.58  3.98  0.16   943.01  1172  937.66   
110000   49  785   99  518.54  796  514.43  3.99  0.16   943.09  1172  937.77   
115000   50  821   59  518.46  795  514.34  4.00  0.15   943.17  1169  937.81   
120000   49  857   19  518.40  794  514.27  4.02  0.15   943.17  1168  937.80   
125000   49  892  119  518.29  793  514.16  4.03  0.15   943.97  1177  938.54   
130000   49  928   79  518.20  792  514.06  4.04  0.15   943.59  1173  938.16   
135000   48  964   39  518.16  791  513.98  4.05  0.14   943.83  1173  938.35   
140000   48  999  139  518.11  790  513.91  4.06  0.14   944.00  1179  938.60   

                     TEST:       SPEC:             HIER:        
          KL   REG    LOSS   PPL     1     2     3 CHILD OTHER  
5000    4.17  0.17  604.89  1133  0.15  0.28  0.38  0.60  0.43  
10000   4.13  0.12  582.45   946  0.21  0.38  0.34  0.47  0.33  
15000   4.48  0.08  580.47   937  0.22  0.31  0.41  0.43  0.29  
20000   4.62  0.22  580.47   937  0.20  0.28  0.32  0.54  0.41  
25000   4.82  0.20  580.47   937  0.22  0.33  0.27  0.51  0.42  
30000   4.50  0.12  580.47   937  0.21  0.31  0.38  0.42  0.35  
35000   3.58  0.10  580.47   937  0.16  0.27  0.22  0.54  0.45  
40000   4.55  0.17  580.47   937  0.20  0.32  0.32  0.45  0.38  
45000   4.57  0.24  580.47   937  0.21  0.31  0.26  0.50  0.46  
50000   4.72  0.21  576.88   876  0.23  0.28  0.28  0.50  0.46  
55000   4.89  0.19  576.88   876  0.22  0.29  0.30  0.50  0.44  
60000   4.94  0.17  576.88   876  0.23  0.32  0.28  0.47  0.43  
65000   5.03  0.16  576.61   871  0.23  0.29  0.30  0.49  0.44  
70000   5.07  0.15  576.52   872  0.24  0.28  0.32  0.48  0.42  
75000   5.01  0.14  576.52   872  0.24  0.31  0.31  0.48  0.41  
80000   5.07  0.13  576.35   867  0.24  0.30  0.31  0.48  0.43  
85000   5.13  0.12  576.18   867  0.24  0.27  0.32  0.48  0.43  
90000   5.22  0.12  576.18   867  0.23  0.28  0.32  0.48  0.42  
95000   5.24  0.11  576.18   867  0.24  0.31  0.31  0.46  0.41  
100000  5.31  0.11  576.18   867  0.24  0.29  0.31  0.48  0.43  
105000  5.25  0.10  576.18   867  0.24  0.28  0.32  0.48  0.43  
110000  5.23  0.10  576.18   867  0.24  0.30  0.31  0.47  0.41  
115000  5.26  0.09  576.18   867

0 R: 1.000 P: 0.142 space use write new article one know university please also
   2 R: 0.034 P: 0.004 use file window program image get version run server available
     21 R: 0.029 P: 0.029 key use chip encryption system one privacy security clipper algorithm
   3 R: 0.194 P: 0.017 use drive get card one problem work write system disk
     31 R: 0.138 P: 0.138 write get people government article one think right make know
     32 R: 0.039 P: 0.039 game team play go year player get win write season
   4 R: 0.629 P: 0.084 get write article one car go like use take think
     41 R: 0.545 P: 0.545 one god say write people think know make see believe
